In [1]:
import pandas as pd
from bs4 import BeautifulSoup

import re
import requests

In [2]:
up_to_trump = pd.read_csv('presidential_speeches.csv')
up_to_trump.tail(10)

,Date,President,Party,Speech Title,Summary,Transcript,URL
982,2018-02-15,Donald Trump,Republican,"Statement on the School Shooting in Parkland, ...",President Trump makes a statement about the sc...,"My fellow Americans, today I speak to a nation...",https://millercenter.org/the-presidency/presid...
983,2018-02-23,Donald Trump,Republican,Remarks at the Conservative Political Action C...,President Trump addresses the Conservative Pol...,Thank you very much. Thank you everybody. Than...,https://millercenter.org/the-presidency/presid...
984,2018-03-19,Donald Trump,Republican,Remarks on Combating the Opioid Crisis,President Trump speaks in New Hampshire about ...,"Thank you to our First Lady, Melania, who has ...",https://millercenter.org/the-presidency/presid...
985,2018-07-24,Donald Trump,Republican,Speech at the Veterans of Foreign Wars Nationa...,President Trump addresses American veterans fr...,"Thank you, Lee. Thank you, Lee. Thank you. And...",https://millercenter.org/the-presidency/presid...
986,2018-09-25,Donald Trump,Republican,Address at the 73rd Session of the United Nati...,President Donald Trump addresses the General A...,"Madam President, Mr. Secretary-General, world ...",https://millercenter.org/the-presidency/presid...
987,2019-01-19,Donald Trump,Republican,Remarks about the US Southern Border,President Donald Trump speaks about what he se...,"Just a short time ago, I had the honor of pres...",https://millercenter.org/the-presidency/presid...
988,2019-02-05,Donald Trump,Republican,State of the Union Address,"In his second State of the Union Address, Pres...","Madam Speaker, Mr. Vice President, Members of ...",https://millercenter.org/the-presidency/presid...
989,2019-02-15,Donald Trump,Republican,Speech Declaring a National Emergency,President Donald Trump declares a national eme...,"Thank you very much, everybody. Before we begi...",https://millercenter.org/the-presidency/presid...
990,2019-09-24,Donald Trump,Republican,Remarks at the United Nations General Assembly,President Donald Trump speaks to the 74th sess...,"Thank you very much. Mr. President, Mr. Secret...",https://millercenter.org/the-presidency/presid...
991,2019-09-25,Donald Trump,Republican,Press Conference,President Donald Trump holds a press conferenc...,"Thank you very much. Thank you. Well, thank yo...",https://millercenter.org/the-presidency/presid...


In [64]:
root_url = 'https://millercenter.org'
page_url = f'{root_url}/the-presidency/presidential-speeches'
result = requests.get(page_url)
content = result.text
soup = BeautifulSoup(content, 'html')

titles_fields = soup.select('div.views-field-title')

links = []

for title_field in titles_fields:
    for link in title_field.find_all('a', href=True):
        links.append(link['href'])
print(links)

['/the-presidency/presidential-speeches/november-15-2021-signing-infrastructure-investment-and-jobs-0', '/the-presidency/presidential-speeches/september-9-2021-remarks-fighting-covid-19-pandemic', '/the-presidency/presidential-speeches/august-26-2021-statement-terror-attacks-afghanistan', '/the-presidency/presidential-speeches/august-16-2021-remarks-situation-afghanistan', '/the-presidency/presidential-speeches/july-8-2021-speech-drawdown-us-forces-afghanistan', '/the-presidency/presidential-speeches/june-1-2021-remarks-commemorating-100th-anniversary-tulsa-race', '/the-presidency/presidential-speeches/april-28-2021-address-joint-session-congress', '/the-presidency/presidential-speeches/march-31-2021-announcing-american-jobs-plan', '/the-presidency/presidential-speeches/march-25-2021-first-press-conference', '/the-presidency/presidential-speeches/march-11-2021-remarks-anniversary-coronavirus-shutdown', '/the-presidency/presidential-speeches/january-20-2021-inaugural-address', '/the-pre

In [76]:
for link in links:
    result = requests.get(f'{root_url}/{link}')
    content = result.text
    soup = BeautifulSoup(content, 'html')

    title = soup.find('h2', class_='presidential-speeches--title').get_text()
    title = title.replace(' ', '_').replace(':', '').replace('\n', '')
    container = soup.find(id='dp-expandable-text')
    paragraphs = container.find_all('p')
    transcript = [paragraph.get_text(strip=True, separator=' ') for paragraph in paragraphs]

    with open('biden_' + f'{title}' + '.txt', 'w', encoding='utf8') as file:
        for line in transcript:
            file.write(f'{line} ')

In [80]:
up_to_trump.columns

Index(['Date', 'President', 'Party', 'Speech Title', 'Summary', 'Transcript',
       'URL'],
      dtype='object')

In [89]:
import os
import datetime
import numpy as np

date_pattern = r'(\d+) (\w+) (\d+)'
# date_format = datetime.datetime.strptime(date_pattern, '%d %B %Y')

df_cols = up_to_trump.columns.values
df_rows = np.zeros(11, dtype='int')

df_biden = dict(zip(df_cols, df_rows))
df_biden['President'] = list(['Joe Biden'] * len(df_rows))
df_biden['Party'] = list(['Democrat'] * len(df_rows))
speeches_dates = []
speeches_urls = []
speeches_transcripts = []
speeches_titles = []

for file in os.listdir('./Biden'):
    filename = os.path.basename(file)
    
    # Extract date
    speech_date_find = re.findall(date_pattern, filename, flags=re.I)
    speech_dates = [' '.join(speech_date_tup) for speech_date_tup in speech_date_find]
    for speech_date in speech_dates:
        speech_date_str = ' '.join(speech_dates)
        speech_date = datetime.datetime.strptime(speech_date_str, '%d %B %Y').date()
        speeches_dates.append(str(speech_date))
    # speech_dates_list = sorted(speech_dates_list)
    
    # Extract title
    find_title = filename.split(str(speech_date_str) + ' - ')
    find_title.remove('')
    plain_title = [title.rsplit('.', 1)[0] for title in find_title]
    speech_title = plain_title[0]
    speeches_titles.append(speech_title)

    # Extract transcript
    with open(os.path.join('./Biden', file), 'r', encoding='utf8') as speech:
        speech_transcript = speech.read()
        speeches_transcripts.append(speech_transcript)

# Extract URL
for link in links[:-1]: # Up to the second to last because last speech is Trump's
    speech_url = root_url + link
    speeches_urls.append(speech_url)
    

df_biden['Date'] = list(speeches_dates)
df_biden['Speech Title'] = list(speeches_titles)
df_biden['Transcript'] = list(speeches_transcripts)
df_biden['URL'] = list(speeches_urls)

biden_df = pd.DataFrame(df_biden)
biden_df.head(15)

,Date,President,Party,Speech Title,Summary,Transcript,URL
0,2021-06-01,Joe Biden,Democrat,Remarks Commemorating the 100th Anniversary of...,0,"THE PRESIDENT: Well, thank you. Please, if you...",https://millercenter.org/the-presidency/presid...
1,2021-03-11,Joe Biden,Democrat,Remarks on the Anniversary of the Coronavirus ...,0,"THE PRESIDENT: Good evening, my fellow America...",https://millercenter.org/the-presidency/presid...
2,2021-11-15,Joe Biden,Democrat,Signing the Infrastructure Investment and Jobs...,0,"THE PRESIDENT: Thank you, thank you, thank you...",https://millercenter.org/the-presidency/presid...
3,2021-08-16,Joe Biden,Democrat,Remarks on Situation in Afghanistan,0,THE PRESIDENT: Good afternoon. I want to speak...,https://millercenter.org/the-presidency/presid...
4,2021-01-20,Joe Biden,Democrat,Inaugural Address,0,"Chief Justice Roberts, Vice President Harris, ...",https://millercenter.org/the-presidency/presid...
5,2021-03-25,Joe Biden,Democrat,First Press Conference,0,"THE PRESIDENT: Please, please sit down. Thank ...",https://millercenter.org/the-presidency/presid...
6,2021-08-26,Joe Biden,Democrat,Statement on Terror Attacks in Afghanistan,0,THE PRESIDENT: Been a tough day. This evening ...,https://millercenter.org/the-presidency/presid...
7,2021-04-28,Joe Biden,Democrat,Address to Joint Session of Congress,0,THE PRESIDENT: Thank you. (Applause.) Thank yo...,https://millercenter.org/the-presidency/presid...
8,2021-03-31,Joe Biden,Democrat,Announcing the American Jobs Plan,0,THE PRESIDENT: Thank you. (Applause.) (Inaudib...,https://millercenter.org/the-presidency/presid...
9,2021-07-08,Joe Biden,Democrat,Speech on the Drawdown of US Forces in Afghani...,0,"THE PRESIDENT: Good afternoon. Earlier today, ...",https://millercenter.org/the-presidency/presid...


In [90]:
biden_df.to_csv('biden-speeches.csv', sep=',', encoding='utf8', index=False)

In [91]:
combo_df = up_to_trump.append(biden_df, ignore_index=True)
combo_df.tail()

,Date,President,Party,Speech Title,Summary,Transcript,URL
998,2021-08-26,Joe Biden,Democrat,Statement on Terror Attacks in Afghanistan,0,THE PRESIDENT: Been a tough day. This evening ...,https://millercenter.org/the-presidency/presid...
999,2021-04-28,Joe Biden,Democrat,Address to Joint Session of Congress,0,THE PRESIDENT: Thank you. (Applause.) Thank yo...,https://millercenter.org/the-presidency/presid...
1000,2021-03-31,Joe Biden,Democrat,Announcing the American Jobs Plan,0,THE PRESIDENT: Thank you. (Applause.) (Inaudib...,https://millercenter.org/the-presidency/presid...
1001,2021-07-08,Joe Biden,Democrat,Speech on the Drawdown of US Forces in Afghani...,0,"THE PRESIDENT: Good afternoon. Earlier today, ...",https://millercenter.org/the-presidency/presid...
1002,2021-09-09,Joe Biden,Democrat,Remarks on Fighting the COVID-⁠19 Pandemic,0,"THE PRESIDENT: Good evening, my fellow America...",https://millercenter.org/the-presidency/presid...


In [93]:
combo_df.to_csv('./presidential_speeches_biden.csv', sep=',', encoding='utf8', index=False)